<a href="https://colab.research.google.com/github/Dicere/WB_Internship/blob/main/0_7_Chek_new_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [16]:
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from catboost import CatBoostClassifier, Pool
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report, recall_score, precision_recall_curve,precision_score, precision_recall_fscore_support
import statsmodels.api as sm
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/content/drive/MyDrive/WB_стажировка/wb_school_task_2.csv.gzip',compression='gzip')
df = df.drop_duplicates(ignore_index=True)

In [4]:
kvad_sum=(lambda x: sum(i**2 for i in x))
kvad_sum.__name__ = 'kvad_sum'

interkvartil_razmah = (lambda x: np.percentile(x, 75) - np.percentile(x, 25))
interkvartil_razmah.__name__ = 'interkvartil_razmah'

range_dannih = (lambda x: x.max()-x.min())
range_dannih.__name__ = 'range_dannih'

quant90 = lambda x: x.quantile(0.9)
quant90.__name__ = 'quant90'

In [5]:
aggs_list = ['min','max', 'mean', 'median', 'count', 'sum',kvad_sum,interkvartil_razmah,range_dannih,quant90]

In [6]:
df_agg = df[['id3','f1','f2','f3','f4','f5','f6','f7','f8']].groupby(by=['id3']).agg(aggs_list).reset_index()
df_agg.columns = ['_by_id3_'.join(col) for col in df_agg.columns.values]
df_agg = df_agg.rename(columns={'id3_by_id3_':'id3'})

In [7]:
df1 = pd.merge(df,df_agg,how='left',on=['id3'])

In [8]:
clear_df = df1[['f1','f2','f3','f4','f5','f6','f7','f8','f1_by_id3_sum', 'f2_by_id3_sum', 'f3_by_id3_sum', 'f4_by_id3_sum',
       'f5_by_id3_sum', 'f6_by_id3_sum', 'f7_by_id3_sum', 'f8_by_id3_sum']]

In [9]:
feature_dict = {}
def summ(b=0,i=0):
  name = str('sum_'+clear_df.columns[b] +"_"+ clear_df.columns[i])
  feature_dict[name] = clear_df.iloc[:,b] + clear_df.iloc[:,i]
def subtraction(b=0,i=0):
    name = str('subtraction_'+clear_df.columns[b] +"_"+ clear_df.columns[i])
    feature_dict[name] = abs(clear_df.iloc[:,b] - clear_df.iloc[:,i])
def multiplication(b=0,i=0):
    name = str('multiplication_'+clear_df.columns[b] +"_"+ clear_df.columns[i])
    feature_dict[name] = abs(clear_df.iloc[:,b] * clear_df.iloc[:,i])
def division(b=0,i=0):
    name = str('division_'+clear_df.columns[b] +"_"+ clear_df.columns[i])
    feature_dict[name] = abs((clear_df.iloc[:,b] / clear_df.iloc[:,i]).fillna(0))
    name = str('division_'+clear_df.columns[i]+"_"+clear_df.columns[b])
    feature_dict[name] = abs((clear_df.iloc[:,i] /clear_df.iloc[:,b]).fillna(0))

In [10]:
conversions_list = {'sum':summ,'subtraction':subtraction,'division':division,'multiplication':multiplication}

In [11]:
idx=0
for conv in conversions_list.keys():
  for b in range(len(clear_df.columns)-1):
    if conv == 'multiplication':
      for i in range(b,len(clear_df.columns)):
        conversions_list[conv](b,i)
    else:
      for i in range(b+1,len(clear_df.columns)):
        conversions_list[conv](b,i)


In [ ]:
df1 = df1.assign(**feature_dict)

In [13]:
cols_to_drop = df1.columns[df1.nunique() == 1]
df1 = df1.drop(cols_to_drop, axis=1)
df1.replace([np.inf, -np.inf], 0, inplace=True)

In [14]:
X = df1[df1.drop('label', axis=1).columns[4::]]
y=df1.label

In [32]:
N_SPLITS = 7
current_feat = list(X.columns[:8])

In [36]:
def test_features(columns,N_SPLITS):
  cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=22)
  cv_scores_AUC_ROC = np.empty(N_SPLITS)
  cv_scores_f1_score = np.empty(N_SPLITS)
  cv_scores_Precision = np.empty(N_SPLITS)
  cv_scores_Recall = np.empty(N_SPLITS)
  cv_scores = np.empty(N_SPLITS)
  roc_auc_score_1 = np.empty(N_SPLITS)
  for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    # print('--'*10+f'TRIAL {trial.number} ~ '+f'FOLD {idx}'+'--'*10)
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val  = y.iloc[train_idx], y.iloc[test_idx]
    train_pool = Pool(X_train[columns], y_train, feature_names=columns)
    test_pool = Pool(X_val[columns], y_val, feature_names=columns)


    model = CatBoostClassifier()
    model.fit(
        train_pool,
        eval_set=test_pool,
        early_stopping_rounds=100,
        use_best_model=True,
        verbose=0

    )
    y_pred_metki = model.predict(test_pool)
    y_pred = model.predict_proba(test_pool)[:, 1]
    precision = precision_score(y_val, y_pred_metki)
    recall = recall_score(y_val, y_pred_metki)
    cv_scores_AUC_ROC[idx] = roc_auc_score(y_val, y_pred)
    cv_scores_f1_score[idx] = f1_score(y_val, y_pred_metki,average='macro')
    # print(precision)
    cv_scores_Precision[idx] = precision
    cv_scores_Recall[idx] = recall
  print("AUC_ROC----"+str(np.mean(cv_scores_AUC_ROC)))
  print("F1-score-----"+str(np.mean(cv_scores_f1_score)))
  print("Precision-----"+str(np.mean(cv_scores_Precision)))
  print("Recall-----"+str(np.mean(cv_scores_Recall)))

### Стартовые фичи

In [37]:
test_features(current_feat,N_SPLITS)

AUC_ROC----0.7497612183590228
F1-score-----0.6521688420618404
Precision-----0.7418994446413801
Recall-----0.30912025827280065


### +1 фича

In [39]:
columns= ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f2_by_id3_mean']

In [40]:
test_features(columns,N_SPLITS)

AUC_ROC----0.758623261816581
F1-score-----0.6540957491218313
Precision-----0.7333994720280212
Recall-----0.31394217834895805


### +2 фича

In [45]:
columns = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'division_f1_f2', 'division_f7_f3_by_id3_sum']

In [46]:
test_features(columns,N_SPLITS)

AUC_ROC----0.7613993723510063
F1-score-----0.651405421439226
Precision-----0.7161741471425598
Recall-----0.3139525258169326


### +3 фичи

In [47]:
columns = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'sum_f3_f6', 'division_f2_f1', 'division_f7_f6_by_id3_sum']

In [48]:
test_features(columns,N_SPLITS)

AUC_ROC----0.7567801289600739
F1-score-----0.6433384366493404
Precision-----0.7096717637824368
Recall-----0.3006456820016142


### 32 feature

In [34]:
columns = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f1_by_id3_median', 'f2_by_id3_mean', 'f2_by_id3_median', 'f6_by_id3_min', 'f6_by_id3_median', 'f6_by_id3_interkvartil_razmah', 'sum_f3_f6', 'subtraction_f3_f6', 'subtraction_f3_by_id3_sum_f6_by_id3_sum', 'division_f2_f1', 'division_f2_f7', 'division_f3_f4', 'division_f4_f7', 'division_f7_f4', 'division_f8_f4', 'division_f7_f5', 'division_f7_f8', 'division_f2_by_id3_sum_f3_by_id3_sum', 'division_f2_by_id3_sum_f6_by_id3_sum', 'division_f2_by_id3_sum_f7_by_id3_sum', 'division_f7_by_id3_sum_f3_by_id3_sum', 'multiplication_f3_f3', 'multiplication_f3_f6', 'multiplication_f4_f7']

In [38]:
test_features(columns,N_SPLITS)

AUC_ROC----0.7688250026308988
F1-score-----0.663151346621588
Precision-----0.7226529021308832
Recall-----0.3369756420603878


### 52 feature

In [42]:
columns =  ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f1_by_id3_median', 'f1_by_id3_interkvartil_razmah', 'f2_by_id3_mean', 'f2_by_id3_quant90', 'f3_by_id3_min', 'f3_by_id3_max', 'f6_by_id3_min', 'f6_by_id3_max', 'f6_by_id3_mean', 'f6_by_id3_median', 'f6_by_id3_interkvartil_razmah', 'f6_by_id3_range_dannih', 'f6_by_id3_quant90', 'f7_by_id3_mean', 'sum_f3_f6', 'sum_f4_f6', 'subtraction_f1_f7', 'subtraction_f2_f6', 'subtraction_f3_f6', 'subtraction_f4_f5', 'subtraction_f3_by_id3_sum_f6_by_id3_sum', 'division_f1_f2', 'division_f6_f3', 'division_f5_f4', 'division_f4_f7', 'division_f7_f4', 'division_f8_f4', 'division_f4_f4_by_id3_sum', 'division_f4_by_id3_sum_f4', 'division_f7_f5', 'division_f5_f2_by_id3_sum', 'division_f7_f8', 'division_f7_f3_by_id3_sum', 'division_f6_by_id3_sum_f7', 'division_f2_by_id3_sum_f3_by_id3_sum', 'division_f3_by_id3_sum_f2_by_id3_sum', 'division_f5_by_id3_sum_f2_by_id3_sum', 'division_f2_by_id3_sum_f6_by_id3_sum', 'division_f6_by_id3_sum_f2_by_id3_sum', 'division_f2_by_id3_sum_f7_by_id3_sum', 'division_f7_by_id3_sum_f3_by_id3_sum', 'multiplication_f3_f3', 'multiplication_f3_f6', 'multiplication_f6_f6']

In [44]:
test_features(columns,N_SPLITS)

AUC_ROC----0.7701107862774038
F1-score-----0.6627505781507379
Precision-----0.7413340021318253
Recall-----0.3309223732952547
